<a href="https://colab.research.google.com/github/sag005/spotify-playlist-recommendation/blob/main/ValidationData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd

In [2]:
tracks_data = pd.read_pickle('track_data.pkl')

In [ ]:
tracks_uris = set(tracks_data['track_uri'].to_list())

In [ ]:
def loadPlaylistData(data,cnt):
  under_30_tracks_playlists = [playlist for playlist in data['playlists'] if 15 <= playlist['num_tracks'] <= 30]
  filtered = []
  df = []
  for playlist in under_30_tracks_playlists:
    track_uris = []
    for track in playlist['tracks']:
      track_uris.append(track['track_uri'])
    if set(track_uris).issubset(tracks_uris):
      playlist['tracks'] = track_uris
      filtered.append(playlist)
  return filtered

In [ ]:
# def loadTrackData(data, cnt):
#   tracks_20_playlists = [playlist for playlist in data['playlists'] if 15 <= playlist['num_tracks'] <= 30]
#   df = []
#   for playlist in tracks_20_playlists:
#     p_df = pd.DataFrame(playlist['tracks'], columns=['album_name','album_uri', 'artist_name', 'artist_uri', 'duration_ms', 'track_name', 'pos', 'track_uri'])
#     track_sub = set(p_df['track_uri'].to_list())
#     if track_sub.issubset(tracks_uris):
#       cnt+=1
#       print(cnt)
#       pl_df = pd.DataFrame(playlist)
#       df.append(pl_df)
#   return df

In [ ]:
all_playlists = []
for fileName in ['mpd.slice.835000-835999.json','mpd.slice.834000-834999.json','mpd.slice.833000-833999.json','mpd.slice.832000-832999.json','mpd.slice.839000-839999.json', 'mpd.slice.838000-838999.json', 'mpd.slice.837000-837999.json', 'mpd.slice.836000-836999.json','mpd.slice.844000-844999.json','mpd.slice.844000-844999.json','mpd.slice.844000-844999.json','mpd.slice.844000-844999.json','mpd.slice.840000-840999.json','mpd.slice.841000-841999.json','mpd.slice.842000-842999.json','mpd.slice.843000-843999.json','mpd.slice.844000-844999.json', 'mpd.slice.845000-845999.json', 'mpd.slice.846000-846999.json', 'mpd.slice.847000-847999.json','mpd.slice.848000-848999.json', 'mpd.slice.849000-849999.json', 'mpd.slice.85000-85999.json', 'mpd.slice.850000-850999.json','mpd.slice.851000-851999.json', 'mpd.slice.852000-852999.json', 'mpd.slice.853000-853999.json', 'mpd.slice.854000-854999.json','mpd.slice.855000-855999.json', 'mpd.slice.856000-856999.json']:
  f = open(fileName)
  data = json.load(f)
  cnt = 0
  all_playlists.extend(loadPlaylistData(data, cnt))

In [ ]:
pll_df = pd.DataFrame(all_playlists)

In [ ]:
pll_df.drop_duplicates('pid', inplace=True)

In [ ]:
pll_df.reset_index(inplace=True)

In [ ]:
pll_df

,index,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,0,morning tunes,false,835096,1495497600,24,23,1,"[spotify:track:0cuq829zRNq4AEdmDJA4aw, spotify...",11,5217269,21,NaN
1,1,my mix,false,835122,1508112000,24,20,1,"[spotify:track:4DMKwE2E2iYDKY01C335Uw, spotify...",2,5247202,18,NaN
2,2,vibes,false,835259,1493856000,19,11,1,"[spotify:track:6y6jbcPG4Yn3Du4moXaenr, spotify...",5,4006365,5,NaN
3,3,Drake,false,835262,1480118400,22,11,1,"[spotify:track:5InOp6q2vvx0fShv3bzFLZ, spotify...",13,5003299,3,NaN
4,4,DANCE PARTY,false,835274,1496707200,22,22,1,"[spotify:track:07Tx168RSsUS1HqkDIOZbH, spotify...",2,4713848,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,435,Party,false,856726,1509235200,27,25,1,"[spotify:track:1vNn7rVQT6UtEW1NGH53fT, spotify...",5,5968241,21,NaN
396,436,old music,false,856838,1488067200,30,26,1,"[spotify:track:7pYPpFhFyOwCjf4y7vdJqM, spotify...",4,7165764,22,NaN
397,437,FunKy,false,856865,1485475200,24,23,2,"[spotify:track:5uuJruktM9fMdN9Va0DUMl, spotify...",9,6038833,21,NaN
398,438,Good songs,false,856893,1509062400,19,15,24,"[spotify:track:1UZOjK1BwmwWU14Erba9CZ, spotify...",8,3947123,13,NaN


In [ ]:
from sklearn.model_selection import train_test_split

valid, test = train_test_split(pll_df, test_size=0.3)

In [ ]:
valid.to_pickle("validation_data.pkl")
test.to_pickle("test_data.pkl")

In [ ]:
test_data = pd.read_pickle('test_data.pkl')

In [36]:
valid.drop(['level_0', 'index'], axis=1)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,wedding songs,false,848075,1476403200,20,20,1,"[spotify:track:75p6fgACKYIyIhsoG411SZ, spotify...",4,4340872,20,NaN
1,Motown,false,835562,1437696000,16,16,2,"[spotify:track:4LxIGAVfcQIw0zAQRyFhU8, spotify...",3,2925429,12,NaN
2,Rap,false,842381,1461888000,17,15,2,"[spotify:track:1ID1QFSNNxi0hiZCNcwjUC, spotify...",8,4039881,11,NaN
3,trolls,false,845069,1503100800,17,7,2,"[spotify:track:2986J0T5CGwJGY3aqgANtE, spotify...",2,2995715,9,NaN
4,Kanye,false,839158,1480982400,15,8,1,"[spotify:track:4WSSY5ZnbhVN2PVXcjY1yh, spotify...",4,3724368,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
275,❤️,false,833681,1500249600,25,23,2,"[spotify:track:6yHkPtl6UQ7RjtJLBPzbJw, spotify...",13,5728808,20,NaN
276,CHILL,false,849330,1506038400,24,19,1,"[spotify:track:0CcQNd8CINkwQfe1RDtGV6, spotify...",5,5111066,19,NaN
277,ok,false,834256,1474588800,22,18,1,"[spotify:track:0z9UVN8VBHJ9HdfYsOuuNf, spotify...",2,4792007,14,NaN
278,old hits,false,85608,1507766400,15,15,1,"[spotify:track:2NBQmPrOEEjA8VbeWOQGxO, spotify...",2,3667420,14,NaN


In [78]:
artist_WithHeld_names = []
with_held_songs = []
for index, row in valid.iterrows():
    tracks = row['tracks']
    seed = tracks[:-10 or None]
    withheld = tracks[-10:]
    artists_withHeld = []
    for track in withheld:
      artists_withHeld.append(tracks_data.loc[tracks_data['track_uri']==track]['artist_name'].iloc[0])
    artist_WithHeld_names.append(artists_withHeld)
    with_held_songs.append(withheld)

In [79]:
valid['with_held_tracks'] = with_held_songs
valid['with_held_artists'] = artist_WithHeld_names


In [80]:
valid

,level_0,index,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,with_held_tracks,with_held_artists
0,233,273,wedding songs,false,848075,1476403200,20,20,1,"[spotify:track:75p6fgACKYIyIhsoG411SZ, spotify...",4,4340872,20,NaN,"[spotify:track:1jyddn36UN4tVsJGtaJfem, spotify...","[Ray LaMontagne, Train, Ben Rector, Band of Ho..."
1,8,8,Motown,false,835562,1437696000,16,16,2,"[spotify:track:4LxIGAVfcQIw0zAQRyFhU8, spotify...",3,2925429,12,NaN,"[spotify:track:2Po67bkabDRE2BcquBNdu1, spotify...","[The Supremes, The Jackson 5, Four Tops, Mary ..."
2,165,195,Rap,false,842381,1461888000,17,15,2,"[spotify:track:1ID1QFSNNxi0hiZCNcwjUC, spotify...",8,4039881,11,NaN,"[spotify:track:10I3CmmwT0BkOVhduDy53o, spotify...","[Kevin Gates, J. Cole, Wiz Khalifa, G-Eazy, Dr..."
3,181,221,trolls,false,845069,1503100800,17,7,2,"[spotify:track:2986J0T5CGwJGY3aqgANtE, spotify...",2,2995715,9,NaN,"[spotify:track:1w1kzejjmiMhdWAOecgo4l, spotify...","[Ariana Grande, Anna Kendrick, Justin Timberla..."
4,57,57,Kanye,false,839158,1480982400,15,8,1,"[spotify:track:4WSSY5ZnbhVN2PVXcjY1yh, spotify...",4,3724368,3,NaN,"[spotify:track:6C7RJEIUDqKkJRZVWdkfkH, spotify...","[Kanye West, Kanye West, Kanye West, Kanye Wes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,38,38,❤️,false,833681,1500249600,25,23,2,"[spotify:track:6yHkPtl6UQ7RjtJLBPzbJw, spotify...",13,5728808,20,NaN,"[spotify:track:3pndPhlQWjuSoXhcIIdBjv, spotify...","[Russ, Rihanna, Migos, Luis Fonsi, Big Sean, D..."
276,251,291,CHILL,false,849330,1506038400,24,19,1,"[spotify:track:0CcQNd8CINkwQfe1RDtGV6, spotify...",5,5111066,19,NaN,"[spotify:track:3A7qX2QjDlPnazUsRk5y0M, spotify...","[David Guetta, Fifth Harmony, Rita Ora, Selena..."
277,20,20,ok,false,834256,1474588800,22,18,1,"[spotify:track:0z9UVN8VBHJ9HdfYsOuuNf, spotify...",2,4792007,14,NaN,"[spotify:track:3pzjHKrQSvXGHQ98dx18HI, spotify...","[Justin Bieber, Rihanna, Beyoncé, Maroon 5, Ju..."
278,275,315,old hits,false,85608,1507766400,15,15,1,"[spotify:track:2NBQmPrOEEjA8VbeWOQGxO, spotify...",2,3667420,14,NaN,"[spotify:track:5XJJdNPkwmbUwE79gv0NxK, spotify...","[Kanye West, Lil Jon & The East Side Boyz, Nel..."


In [81]:
artist_WithHeld_names = []
with_held_songs = []
for index, row in test.iterrows():
    tracks = row['tracks']
    seed = tracks[:-10 or None]
    withheld = tracks[-10:]
    artists_withHeld = []
    for track in withheld:
      artists_withHeld.append(tracks_data.loc[tracks_data['track_uri']==track]['artist_name'].iloc[0])
    artist_WithHeld_names.append(artists_withHeld)
    with_held_songs.append(withheld)

In [82]:
test['with_held_tracks'] = with_held_songs
test['with_held_artists'] = artist_WithHeld_names

In [83]:
test

,index,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,with_held_tracks,with_held_artists
87,87,rap,false,838952,1472169600,15,15,1,"[spotify:track:6fujklziTHa8uoM5OQSfIo, spotify...",6,3468169,14,NaN,"[spotify:track:5SsR3wtCOafDmZgvIdRhSm, spotify...","[ScHoolboy Q, DRAM, Snoop Dogg, Denzel Curry, ..."
35,35,Guardians,false,833559,1503705600,21,20,1,"[spotify:track:5IMtdHjJ1OtkxbGe4zfUxQ, spotify...",4,5147061,19,NaN,"[spotify:track:7fqhrLJzKHJ0RW32N0y2Gp, spotify...","[Norman Greenbaum, Redbone, The Five Stairstep..."
167,197,turnt,false,842628,1468800000,19,17,2,"[spotify:track:0PT7nlpo11hYYyfnBgtilT, spotify...",2,3986457,15,NaN,"[spotify:track:1Ser4X0TKttOvo8bgdytTP, spotify...","[Rae Sremmurd, Jidenna, Rihanna, Ginuwine, Dra..."
74,74,✌🏻️,false,838330,1461542400,19,18,1,"[spotify:track:17Fd6Yb7mSbinKG8LoWfFl, spotify...",5,4266350,18,NaN,"[spotify:track:3ge54HUrk1ht3EX5SMhIzZ, spotify...","[Meghan Trainor, Charlie Puth, The Chainsmoker..."
86,86,wubba lubba dub dub,false,838917,1494460800,18,16,2,"[spotify:track:2VzpKY6w1RGr5J0Fz5pmKa, spotify...",4,4024536,15,NaN,"[spotify:track:343YBumqHu19cGoGARUTsd, spotify...","[Drake, Rae Sremmurd, Kendrick Lamar, Kendrick..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,284,2017,false,848703,1496102400,18,16,1,"[spotify:track:4aWmUDTfIPGksMNLV2rQP2, spotify...",11,4030939,14,NaN,"[spotify:track:152lZdxL1OR0ZMW6KquMif, spotify...","[Khalid, A Boogie Wit da Hoodie, Bebe Rexha, L..."
284,324,april,false,850078,1494115200,15,14,3,"[spotify:track:4Q3N4Ct4zCuIHuZ65E3BD4, spotify...",9,3234996,14,NaN,"[spotify:track:6mapJIPnQ23RTAevUoE0DL, spotify...","[Rae Sremmurd, PnB Rock, Lupe Fiasco, NAV, Ken..."
78,78,Popular,false,838429,1479254400,25,24,1,"[spotify:track:7BKLCZ1jbUBVqRi2FVlTVw, spotify...",7,5748439,22,NaN,"[spotify:track:0azC730Exh71aQlOt9Zj3y, spotify...","[Calvin Harris, G-Eazy, Mike Posner, Avicii, F..."
362,402,Oldies,false,854955,1491609600,30,27,1,"[spotify:track:0LN0ASTtcGIbNTnjSHG6eO, spotify...",5,7040543,18,NaN,"[spotify:track:2b9lp5A6CqSzwOrBfAFhof, spotify...","[Ozzy Osbourne, Motörhead, Boston, The Kinks, ..."


In [84]:
test.drop('index', axis=1)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,with_held_tracks,with_held_artists
87,rap,false,838952,1472169600,15,15,1,"[spotify:track:6fujklziTHa8uoM5OQSfIo, spotify...",6,3468169,14,NaN,"[spotify:track:5SsR3wtCOafDmZgvIdRhSm, spotify...","[ScHoolboy Q, DRAM, Snoop Dogg, Denzel Curry, ..."
35,Guardians,false,833559,1503705600,21,20,1,"[spotify:track:5IMtdHjJ1OtkxbGe4zfUxQ, spotify...",4,5147061,19,NaN,"[spotify:track:7fqhrLJzKHJ0RW32N0y2Gp, spotify...","[Norman Greenbaum, Redbone, The Five Stairstep..."
167,turnt,false,842628,1468800000,19,17,2,"[spotify:track:0PT7nlpo11hYYyfnBgtilT, spotify...",2,3986457,15,NaN,"[spotify:track:1Ser4X0TKttOvo8bgdytTP, spotify...","[Rae Sremmurd, Jidenna, Rihanna, Ginuwine, Dra..."
74,✌🏻️,false,838330,1461542400,19,18,1,"[spotify:track:17Fd6Yb7mSbinKG8LoWfFl, spotify...",5,4266350,18,NaN,"[spotify:track:3ge54HUrk1ht3EX5SMhIzZ, spotify...","[Meghan Trainor, Charlie Puth, The Chainsmoker..."
86,wubba lubba dub dub,false,838917,1494460800,18,16,2,"[spotify:track:2VzpKY6w1RGr5J0Fz5pmKa, spotify...",4,4024536,15,NaN,"[spotify:track:343YBumqHu19cGoGARUTsd, spotify...","[Drake, Rae Sremmurd, Kendrick Lamar, Kendrick..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2017,false,848703,1496102400,18,16,1,"[spotify:track:4aWmUDTfIPGksMNLV2rQP2, spotify...",11,4030939,14,NaN,"[spotify:track:152lZdxL1OR0ZMW6KquMif, spotify...","[Khalid, A Boogie Wit da Hoodie, Bebe Rexha, L..."
284,april,false,850078,1494115200,15,14,3,"[spotify:track:4Q3N4Ct4zCuIHuZ65E3BD4, spotify...",9,3234996,14,NaN,"[spotify:track:6mapJIPnQ23RTAevUoE0DL, spotify...","[Rae Sremmurd, PnB Rock, Lupe Fiasco, NAV, Ken..."
78,Popular,false,838429,1479254400,25,24,1,"[spotify:track:7BKLCZ1jbUBVqRi2FVlTVw, spotify...",7,5748439,22,NaN,"[spotify:track:0azC730Exh71aQlOt9Zj3y, spotify...","[Calvin Harris, G-Eazy, Mike Posner, Avicii, F..."
362,Oldies,false,854955,1491609600,30,27,1,"[spotify:track:0LN0ASTtcGIbNTnjSHG6eO, spotify...",5,7040543,18,NaN,"[spotify:track:2b9lp5A6CqSzwOrBfAFhof, spotify...","[Ozzy Osbourne, Motörhead, Boston, The Kinks, ..."


In [90]:
valid.drop(['index', 'level_0'], axis=1)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,with_held_tracks,with_held_artists
0,wedding songs,false,848075,1476403200,20,20,1,"[spotify:track:75p6fgACKYIyIhsoG411SZ, spotify...",4,4340872,20,NaN,"[spotify:track:1jyddn36UN4tVsJGtaJfem, spotify...","[Ray LaMontagne, Train, Ben Rector, Band of Ho..."
1,Motown,false,835562,1437696000,16,16,2,"[spotify:track:4LxIGAVfcQIw0zAQRyFhU8, spotify...",3,2925429,12,NaN,"[spotify:track:2Po67bkabDRE2BcquBNdu1, spotify...","[The Supremes, The Jackson 5, Four Tops, Mary ..."
2,Rap,false,842381,1461888000,17,15,2,"[spotify:track:1ID1QFSNNxi0hiZCNcwjUC, spotify...",8,4039881,11,NaN,"[spotify:track:10I3CmmwT0BkOVhduDy53o, spotify...","[Kevin Gates, J. Cole, Wiz Khalifa, G-Eazy, Dr..."
3,trolls,false,845069,1503100800,17,7,2,"[spotify:track:2986J0T5CGwJGY3aqgANtE, spotify...",2,2995715,9,NaN,"[spotify:track:1w1kzejjmiMhdWAOecgo4l, spotify...","[Ariana Grande, Anna Kendrick, Justin Timberla..."
4,Kanye,false,839158,1480982400,15,8,1,"[spotify:track:4WSSY5ZnbhVN2PVXcjY1yh, spotify...",4,3724368,3,NaN,"[spotify:track:6C7RJEIUDqKkJRZVWdkfkH, spotify...","[Kanye West, Kanye West, Kanye West, Kanye Wes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,❤️,false,833681,1500249600,25,23,2,"[spotify:track:6yHkPtl6UQ7RjtJLBPzbJw, spotify...",13,5728808,20,NaN,"[spotify:track:3pndPhlQWjuSoXhcIIdBjv, spotify...","[Russ, Rihanna, Migos, Luis Fonsi, Big Sean, D..."
276,CHILL,false,849330,1506038400,24,19,1,"[spotify:track:0CcQNd8CINkwQfe1RDtGV6, spotify...",5,5111066,19,NaN,"[spotify:track:3A7qX2QjDlPnazUsRk5y0M, spotify...","[David Guetta, Fifth Harmony, Rita Ora, Selena..."
277,ok,false,834256,1474588800,22,18,1,"[spotify:track:0z9UVN8VBHJ9HdfYsOuuNf, spotify...",2,4792007,14,NaN,"[spotify:track:3pzjHKrQSvXGHQ98dx18HI, spotify...","[Justin Bieber, Rihanna, Beyoncé, Maroon 5, Ju..."
278,old hits,false,85608,1507766400,15,15,1,"[spotify:track:2NBQmPrOEEjA8VbeWOQGxO, spotify...",2,3667420,14,NaN,"[spotify:track:5XJJdNPkwmbUwE79gv0NxK, spotify...","[Kanye West, Lil Jon & The East Side Boyz, Nel..."


In [91]:
# valid.to_pickle('Validation_dataset.pkl')

In [89]:
valid

,level_0,index,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,with_held_tracks,with_held_artists
0,233,273,wedding songs,false,848075,1476403200,20,20,1,"[spotify:track:75p6fgACKYIyIhsoG411SZ, spotify...",4,4340872,20,NaN,"[spotify:track:1jyddn36UN4tVsJGtaJfem, spotify...","[Ray LaMontagne, Train, Ben Rector, Band of Ho..."
1,8,8,Motown,false,835562,1437696000,16,16,2,"[spotify:track:4LxIGAVfcQIw0zAQRyFhU8, spotify...",3,2925429,12,NaN,"[spotify:track:2Po67bkabDRE2BcquBNdu1, spotify...","[The Supremes, The Jackson 5, Four Tops, Mary ..."
2,165,195,Rap,false,842381,1461888000,17,15,2,"[spotify:track:1ID1QFSNNxi0hiZCNcwjUC, spotify...",8,4039881,11,NaN,"[spotify:track:10I3CmmwT0BkOVhduDy53o, spotify...","[Kevin Gates, J. Cole, Wiz Khalifa, G-Eazy, Dr..."
3,181,221,trolls,false,845069,1503100800,17,7,2,"[spotify:track:2986J0T5CGwJGY3aqgANtE, spotify...",2,2995715,9,NaN,"[spotify:track:1w1kzejjmiMhdWAOecgo4l, spotify...","[Ariana Grande, Anna Kendrick, Justin Timberla..."
4,57,57,Kanye,false,839158,1480982400,15,8,1,"[spotify:track:4WSSY5ZnbhVN2PVXcjY1yh, spotify...",4,3724368,3,NaN,"[spotify:track:6C7RJEIUDqKkJRZVWdkfkH, spotify...","[Kanye West, Kanye West, Kanye West, Kanye Wes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,38,38,❤️,false,833681,1500249600,25,23,2,"[spotify:track:6yHkPtl6UQ7RjtJLBPzbJw, spotify...",13,5728808,20,NaN,"[spotify:track:3pndPhlQWjuSoXhcIIdBjv, spotify...","[Russ, Rihanna, Migos, Luis Fonsi, Big Sean, D..."
276,251,291,CHILL,false,849330,1506038400,24,19,1,"[spotify:track:0CcQNd8CINkwQfe1RDtGV6, spotify...",5,5111066,19,NaN,"[spotify:track:3A7qX2QjDlPnazUsRk5y0M, spotify...","[David Guetta, Fifth Harmony, Rita Ora, Selena..."
277,20,20,ok,false,834256,1474588800,22,18,1,"[spotify:track:0z9UVN8VBHJ9HdfYsOuuNf, spotify...",2,4792007,14,NaN,"[spotify:track:3pzjHKrQSvXGHQ98dx18HI, spotify...","[Justin Bieber, Rihanna, Beyoncé, Maroon 5, Ju..."
278,275,315,old hits,false,85608,1507766400,15,15,1,"[spotify:track:2NBQmPrOEEjA8VbeWOQGxO, spotify...",2,3667420,14,NaN,"[spotify:track:5XJJdNPkwmbUwE79gv0NxK, spotify...","[Kanye West, Lil Jon & The East Side Boyz, Nel..."
